In [0]:
print("Healthy")

In [0]:
global pd 
import pandas as pd
import numpy as np 
import duckdb
from re import search
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings('ignore')


from datetime import datetime, timedelta
d = datetime.today() - timedelta(days = 1)
d = d.strftime('%Y-%m-%d')

date = "'" + d + "'"
print(date, d)

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders

import mysql.connector
import json
with open('/Workspace/Credentials/db_data.json', 'r') as fp:
    data = json.load(fp)


Mifosdb = mysql.connector.connect(
  host=data['mifos']['host'],
  user=data['mifos']['user'],
  passwd=data['mifos']['passwd'],
  database = data['mifos']['database']
)

Dumpdb = mysql.connector.connect(
  host=data['Dumpdb']['host'],
  user=data['Dumpdb']['user'],
  passwd=data['Dumpdb']['passwd'],
  database = data['Dumpdb']['database']
)


host = data['redshift']['host']
user = data['redshift']['user']
passwd = data['redshift']['passwd']
database = data['redshift']['database']

conn = create_engine(f"postgresql+psycopg2://{user}:{passwd}@{host}:5439/{database}")
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)


In [0]:

# fetch last day of the month for a range of dates
def get_last_day_of_month(start_date, end_date):
    from datetime import datetime
    date1 = datetime.strptime(start_date, '%Y-%m-%d')
    date2 = datetime.strptime(end_date, '%Y-%m-%d')

    df_dates = pd.date_range(start=f"{date1}", end=f"{date2}").tolist()

    dates1 = [x.strftime('%Y-%m-%d') for x in df_dates]
    dates2 = [x.strftime('%b-%y') for x in df_dates]

    months= sorted(list(set([(x.strftime('%Y-%m'), x.strftime('%Y_%m'), x.strftime('%b_%y')) for x in df_dates])))
    # month_names = set([x.strftime('%b_%Y') for x in df_dates])
    last_days = []
    for dt in months:
        md = dt[0]
        d = dt[1]
        n = dt[2].lower()
        if d.split('_')[1] in ['01', '03', '05', '07', '08', '10', '12']:
            last_days.append((md+"-31", d+"_31", n))
        elif d.split('_')[1] in ['04', '06', '09', '11']:
            last_days.append((md+"-30", d+"_30", n))
        else:
            last_days.append((md+"-28", d+"_28", n))
    
    return last_days

In [0]:
# get days of the month from within a date range
def get_days_of_month(start_date, end_date):
    from datetime import datetime
    date1 = datetime.strptime(start_date, '%Y-%m-%d')
    date2 = datetime.strptime(end_date, '%Y-%m-%d')

    df_dates = pd.date_range(start=f"{date1}", end=f"{date2}").tolist()

    dates1 = [x.strftime('%Y-%m-%d') for x in df_dates]
    dates2 = [x.strftime('%b_%y') for x in df_dates]
    
    joint_dates = [(x.strftime('%Y-%m-%d'), (x.strftime('%Y_%m_%d')).lower()) for x in df_dates]
    
    return joint_dates

In [0]:
dates = get_days_of_month('2024-01-01', '2024-06-30')

## Spooling Deposit balances

In [0]:
# 1. import all accounts opened till the current date of interest from the db
ja = '''
SELECT
ifnull(mc.referred_by_id, 0) as staff_client_id,
ifnull(mc.id, mg.id) AS 'client_id',
replace(ifnull(mc.display_name, mg.display_name), 'null', '') AS 'client_name',
msa.account_no AS 'account_no',
msa.product_id as 'product_id',
msp.name as product_name,
msa.closedon_date,
msa.approvedon_date
from m_savings_account msa
left JOIN m_client mc ON msa.client_id = mc.id
left join m_group mg on mg.id = msa.group_id
left JOIN m_savings_product msp on msp.id = msa.product_id
WHERE left(msa.closedon_date, 4) = '2024'
and msp.id in (24, 26, 29, 30, 31, 32, 33, 36)
and msa.activatedon_date <= '2024-06-30'

UNION

SELECT 
ifnull(mc.referred_by_id, 0) as staff_client_id,
ifnull(mc.id, mg.id) AS 'client_id',
replace(ifnull(mc.display_name, mg.display_name), 'null', '') AS 'client_name',
msa.account_no AS 'account_no',
msa.product_id as 'product_id',
msp.name as product_name,
msa.closedon_date,
msa.approvedon_date
from m_savings_account msa
left JOIN m_client mc ON msa.client_id = mc.id
left join m_group mg on mg.id = msa.group_id
left JOIN m_savings_product msp on msp.id = msa.product_id
WHERE isnull(msa.closedon_date)
and msp.id in (24, 26, 29, 30, 31, 32, 33, 36)
and msa.activatedon_date <= '2024-06-30';
'''

df_ja = pd.read_sql_query(ja, Mifosdb)
df_ja['account_no'] = df_ja['account_no'].astype(str)
df_ja


In [0]:
staff = pd.read_excel("/Workspace/ReportDump/Staff_Performance/Active Staff List Jul-24.xlsx")

# use staff account numbers to fetch their referral codes 
def get_referral_codes(account_list):
    
    accounts = ','.join([str(x) for x in account_list])
    
    mif = f'''select mc.referred_by_id, mc.id as staff_client_id, mc.referral_id as staff_code, mc.display_name as staff_name, ai.`Email Address`, ai.`Mobile Number`, sa.account_no, sa.id account_id, sa.product_id, 
    sp.name as product, sa.account_balance_derived, 
    sa.approvedon_date, sa.closedon_date
    from m_client mc join m_savings_account sa on sa.client_id = mc.id 
    join m_savings_product sp on sp.id = sa.product_id
    left join `Address Info` ai on ai.client_id = mc.id
    where sa.account_no in ({accounts})
    order by 8 desc
    '''
    
    dfmif = pd.read_sql_query(mif, Mifosdb)
    
    return dfmif[['staff_client_id', 'staff_code', 'staff_name']]


df = get_referral_codes(staff['Account Numbers'].tolist())


# staff downlines
staff_dl = pd.merge(df, df_ja, on = 'staff_client_id', how = 'inner')
staff_dl = staff_dl[[col for col in staff_dl.columns if col != 'closedon_date']]
staff_dl

In [0]:
# for Winifred-1013349512
# Referral DTNVG 

# Esther Nwafor
# 1012606371
# KB5QB
#KQWU5 , XAXFW, KB5QB, DTNVG

In [0]:
def compute_avg_balance(start_date, end_date, full_df):
    
    accts = ','.join([str(x) for x in full_df['account_no'].tolist()])
    dates2 = [d[1] for d in get_days_of_month(start_date, end_date)]

    dts = ['"'+x+'"' for x in dates2]
    c = ','.join([str(x) for x in dts])
    
    ttw = f'''
    select
    account_no,
    {c}  
    from twenty_twenty_two_deposit_balances tt2
    where tt2.account_no in ({accts})
        '''

    df_ttw = pd.read_sql_query(ttw, conn)
    df_ttw['account_no'] = df_ttw['account_no'].astype(str)
    
    df_acc = pd.merge(full_df, df_ttw, on = 'account_no', how = 'left')
    final_df = df_acc[['staff_client_id', 'staff_code', 'staff_name', 'client_id', 'client_name', 'account_no', 'product_name', 'approvedon_date']]
    for dt in set([d[:7] for d in dates2]):

        lis = ['account_no']
        lis.extend([col for col in df_acc.columns if dt in col])
        dff = df_acc[lis]
        
        col_name = f'Avg_Deposit_{dt}'
        
        dff[col_name] = dff[[col for col in dff.columns[1:]]].mean(axis=1)

        dff = dff[[f'{dff.columns[0]}', f'{dff.columns[-1]}']]
        final_df = pd.merge(final_df, dff, on = 'account_no', how = 'inner')
        final_df.drop_duplicates(subset = 'account_no',  inplace = True)
        final_df = final_df.reset_index(drop = True)
        
    
    return final_df

In [0]:
%%time
dep_bal = compute_avg_balance('2024-01-01', '2024-06-30', staff_dl)

In [0]:
# Esther and winifred staff codes only
#ew = dep_bal[dep_bal['staff_code'].isin(['GJ6TR'])].reset_index(drop = True) 
#ew.head()

In [0]:
#ew['staff_code'].unique()

In [0]:
#dep_bal.to_excel('Jun-24 Staff Deposit Performance.xlsx', index = False)

In [0]:
dep_bal.shape

## Spooling Loan balances

In [0]:
%%time
la = '''
SELECT 
ifnull(mc.referred_by_id, 0) as staff_client_id,
ifnull(mc.id, mg.id) AS 'client_id',
replace(ifnull(mc.display_name, mg.display_name), 'null', '') as client_name,
ml.account_no AS 'account_no',
mpl.id AS loan_product_id,
mpl.name AS loan_type,
ml.disbursedon_date,
ml.closedon_date
FROM m_loan ml
left JOIN m_client mc ON mc.id = ml.client_id
left join m_group mg on mg.id = ml.group_id
left join m_product_loan mpl ON mpl.id = ml.product_id
WHERE LEFT(ml.closedon_date, 4) = '2024' 
AND !ISNULL(ml.disbursedon_date)
and ml.disbursedon_date <= '2024-06-30'


UNION


SELECT 
ifnull(mc.referred_by_id, 0) as staff_client_id,
ifnull(mc.id, mg.id) AS 'client_id',
replace(ifnull(mc.display_name, mg.display_name), 'null', '') as client_name,
ml.account_no AS 'account_no',
mpl.id AS loan_product_id,
mpl.name AS loan_type,
ml.disbursedon_date,
ml.closedon_date
FROM m_loan ml
JOIN m_client mc ON mc.id = ml.client_id
left join m_group mg on mg.id = ml.group_id
left join m_product_loan mpl ON mpl.id = ml.product_id
WHERE isnull(ml.closedon_date) 
AND !ISNULL(ml.disbursedon_date)
and ml.disbursedon_date <= '2024-06-30';
'''

df_la = pd.read_sql_query(la, Mifosdb)
df_la['account_no'] = df_la['account_no'].astype(str)
df_la

In [0]:
dflo = get_referral_codes(staff['Account Numbers'].tolist())


# staff downlines
stafflo_dl = pd.merge(dflo, df_la, on = 'staff_client_id', how = 'inner')
stafflo_dl.drop_duplicates(subset = 'account_no', inplace = True)
stafflo_dl = stafflo_dl[[col for col in stafflo_dl.columns if col != 'closedon_date']]
stafflo_dl

In [0]:
def compute_loan_balance(start_date, end_date, full_df):
    
    accts = ','.join([str(x) for x in full_df['account_no'].tolist()])
    dates2 = [d[1] for d in get_days_of_month(start_date, end_date)]

    dts = ['"'+x+'"' for x in dates2]
    c = ','.join([str(x) for x in dts])
    
    ttw = f'''
    select
    account_no,
    {c}  
    from twenty_twenty_two_loan_balances tt2
    where tt2.account_no in ({accts})
        '''

    df_ttw = pd.read_sql_query(ttw, conn)
    df_ttw['account_no'] = df_ttw['account_no'].astype(str)
    
    df_acc = pd.merge(full_df, df_ttw, on = 'account_no', how = 'left')
    final_df = df_acc[['staff_client_id', 'staff_code', 'staff_name', 'client_id', 'client_name', 'account_no', 'loan_type', 'disbursedon_date']]
    for dt in set([d[:7] for d in dates2]):

        lis = ['account_no']
        lis.extend([col for col in df_acc.columns if dt in col])
        dff = df_acc[lis]
        
        col_name = f'Avg_Loan_{dt}'
        
        dff[col_name] = dff[[col for col in dff.columns[1:]]].mean(axis=1)

        dff = dff[[f'{dff.columns[0]}', f'{dff.columns[-1]}']]
        final_df = pd.merge(final_df, dff, on = 'account_no', how = 'inner')
        final_df.drop_duplicates(subset = 'account_no',  inplace = True)
        final_df = final_df.reset_index(drop = True)
        
    
    return final_df

In [0]:
%%time
lo_bal = compute_loan_balance('2024-01-01', '2024-06-30', stafflo_dl)

In [0]:
#ewlo = lo_bal[lo_bal['staff_code'].isin(['GJ6TR'])].reset_index(drop = True) 
#ewlo.head()

In [0]:
#s_code = ewlo['staff_code'].unique()[0]
#s_code

In [0]:
#lo_bal.to_excel('Feb-24 Staff Loan Performance.xlsx', index = False)

In [0]:
lo_bal.shape

In [0]:
#with pd.ExcelWriter(f"C:/Users/babatunde.omotayo/Downloads/V_bank_tasks/projects/staff performance/{s_code} Jan-Apr 2024.xlsx") as writer:
#    ew.to_excel(writer, sheet_name = 'deposits', index = False)
#    ewlo.to_excel(writer, sheet_name = 'loans', index = False)

In [0]:
with pd.ExcelWriter(f"/Workspace/ReportDump/Staff_Performance/V4 Staff Financial performance H1-24.xlsx") as writer:
    dep_bal.to_excel(writer, sheet_name = 'deposits', index = False)
    lo_bal.to_excel(writer, sheet_name = 'loans', index = False)

In [0]:
# use staff referral codes to fetch their downline accounts 
def get_accounts(referral_code):
    rf = pd.read_sql_query(f'''select id, display_name from m_client 
    where referral_id = '{referral_code}'
    ''', Mifosdb)
    
    referred_by_id = rf['id'][0]
    
    mif = f'''select "{referral_code}" as staff_code, mc.display_name as client_id, sa.account_no, sa.id as account_id, sa.product_id, sp.name as product_name, 
    sa.account_balance_derived, sa.approvedon_date as activation_date,
    sa.closedon_date, referred_by_id
    from m_client mc 
    join m_savings_account sa on sa.client_id = mc.id 
    join m_savings_product sp on sp.id = sa.product_id
    where mc.referred_by_id = '{referred_by_id}'
    and left(sa.approvedon_date, 7) >= '2023-01'
    '''
    
    dfmif = pd.read_sql_query(mif, Mifosdb)
    
    
    lo = f'''select "{referral_code}" as staff_code, mc.display_name as client_name, sa.account_no, sa.id as loan_id, sa.product_id, sp.name as product_name, 
    sa.principal_disbursed_derived as principal_disbursed, sa.principal_outstanding_derived as principal_outstanding, sa.disbursedon_date as disbursement_date,
    sa.closedon_date, referred_by_id
    from m_client mc 
    left join m_loan sa on sa.client_id = mc.id 
    join m_product_loan sp on sp.id = sa.product_id
    where mc.referred_by_id = '{referred_by_id}'
    and left(sa.disbursedon_date, 7) >= '2023-01'
    '''
    
    dflo = pd.read_sql_query(lo, Mifosdb)
    
    return rf, dfmif, dflo

# df = get_referrals('JTZLM')


In [0]:
# comput average monthly balance of deposit account lists
def compute_avg_deposit_bal(account_df, start_date, end_date):
    account_list = account_df['account_no'].tolist()
    account_df['account_no'] = account_df['account_no'].astype('int64')
    
    dates = get_last_day_of_month(start_date, end_date)

    dep_bal = pd.DataFrame(columns = ['account_no'])
    for dt in dates:
        d = dt[0]
        n = dt[1]
        try:
            temp = pd.read_sql_query(f'''
                            select
                            account_no,
                            "{d}" as "{n}"
                            from twenty_twenty_two_deposit_balances
                            where account_no in ({','.join([str(x) for x in account_list])})
                            ''', conn)
            if len(dep_bal) < len(temp):
                dep_bal = pd.merge(temp, dep_bal, on = 'account_no', how = 'left')
            else:
                dep_bal = pd.merge(dep_bal, temp, on = 'account_no', how = 'left')

        except Exception as e:
            continue


    import math
    avgd = []
    values = dep_bal.iloc[:, 1:]
    for column in values.columns:
        values[column] = values[column].astype(float)
        
    for row in values.itertuples():
        row = [i for i in row][1:]
        items = [a for a in row if math.isnan(a) == False]
        nulls = [a for a in row if math.isnan(a) == True]

        try:
            avg = sum(items)/len(items)
        except ZeroDivisionError:
            avg = np.nan
        avgd.append(avg)

    dep_bal['average_monthly_balance'] = avgd
    dep_bal['account_no'] = dep_bal['account_no'].astype('int64')
    
    
    dep_bal = pd.merge(dep_bal, account_df[['staff_code', 'account_no']], on = 'account_no', how = 'left')
    
    return dep_bal


In [0]:
# compute average monthly balance of loan account lists
def compute_avg_loan_bal(account_df, start_date, end_date):
    loan_account_list = account_df['account_no'].tolist()
    
    account_df['account_no'] = account_df['account_no'].astype('int64')
    
    dates = get_last_day_of_month(start_date, end_date)

    lon_bal = pd.DataFrame(columns = ['account_no'])
    for dt in dates:
        d = dt[0]
        n = dt[1]
        try:
            temp = pd.read_sql_query(f'''
                            select
                            account_no,
                            "{d}" as "{n}"
                            from twenty_twenty_two_loan_balances
                            where account_no in ({','.join([str(x) for x in loan_account_list])})
                            ''', conn)
            if len(lon_bal) < len(temp):
                lon_bal = pd.merge(temp, lon_bal, on = 'account_no', how = 'left')
            else:
                lon_bal = pd.merge(lon_bal, temp, on = 'account_no', how = 'left')

        except Exception as e:
            continue


    import math
    avgd = []
    values = lon_bal.iloc[:, 1:]
    for column in values.columns:
        values[column] = values[column].astype(float)
    for row in values.itertuples():
        row = [i for i in row][1:]
        items = [a for a in row if math.isnan(a) == False]
        nulls = [a for a in row if math.isnan(a) == True]

        try:
            avg = sum(items)/len(items)
        except ZeroDivisionError:
            avg = np.nan
        avgd.append(avg)

    lon_bal['average_monthly_balance'] = avgd
    lon_bal['account_no'] = lon_bal['account_no'].astype('int64')
    
    lon_bal = pd.merge(lon_bal, account_df[['staff_code', 'account_no']], on = 'account_no', how = 'left')
    
    return lon_bal


In [0]:
def monthly_agg_balance(staff, df):
    staff = staff.upper()
    condition = lambda x: x != 0
    sums = []
    counts = []
    months = []
    for col in df.columns:
        if col not in (['client_id', 'account_no', 'average_monthly_balance']):
            df[col] = df[col].fillna(0)
            total = df[col].sum()
            sums.append(total)
            ct = len(list(filter(condition, df[col])))
            counts.append(ct)
            months.append(col[:7])
            
            
    output = [(a, i, j) for a, i, j in zip(months, sums, counts)]
    dfout = pd.DataFrame(output, columns = ['month', 'total_value', 'total_count'])
    dfout['staff_code'] = staff
    
    return dfout

In [0]:
for staff in staff_list[:]:
    df = get_referrals(staff)
    dep = df[1]
    lo = df[2]

    lon_bal = compute_avg_loan_bal(lo, '2023-01-01', '2023-06-30')
    dep_bal = compute_avg_deposit_bal(dep, '2023-01-01', '2023-06-30')

In [0]:
lon_bal

In [0]:
dep_bal

In [0]:
# All staff figures appended on the same table 
def all_staff(staff_list, start_date, end_date):
    col_headers = ['account_no']
    col_headers.extend([i[1] for i in get_last_day_of_month(start_date, end_date)])
    col_headers.extend(['average_monthly_balance', 'staff_code'])

    loan_data = pd.DataFrame(columns = col_headers)
    deposit_data = pd.DataFrame(columns = col_headers)
    
    full_loan = pd.DataFrame(columns = ['staff_code', 'month', 'total_value', 'total_count'])

    full_deposit = pd.DataFrame(columns = ['staff_code', 'month', 'total_value', 'total_count'])

    for staff in staff_list:
        df = get_referrals(staff)
        dep = df[1]
        lo = df[2]

        lon_bal = compute_avg_loan_bal(lo, start_date, end_date)
        dep_bal = compute_avg_deposit_bal(dep, start_date, end_date)
        
        # this generates the raw data containing individual loan accounts and their corresponding month end balances referred by each staff
        loan_data = pd.concat([loan_data, lon_bal], ignore_index = True)
        # this generates the raw data containing individual deposit accounts and their corresponding month end balances referred by each staff
        deposit_data = pd.concat([deposit_data, dep_bal], ignore_index = True)
        
        
        loans = monthly_agg_balance(staff, lon_bal)

        deposits = monthly_agg_balance(staff, dep_bal)

        full_loan = pd.concat([full_loan, loans], ignore_index = True)

        full_deposit = pd.concat([full_deposit, deposits], ignore_index = True)
        
        
    
    return loan_data, deposit_data

In [0]:
%%time
tst = all_staff(staff_list, '2023-01-01', '2023-06-30')

In [0]:
l = tst[0]
l

In [0]:
d = tst[1]
d

In [0]:
loclt = pd.read_sql_query(f'''
                select
                ml.client_id,
                mc.display_name as client_name,
                ml.account_no,
                mpl.name as product_name,
                ml.closedon_date
                from m_loan ml
                left join m_client mc on mc.id = ml.client_id
                left join m_product_loan mpl on mpl.id = ml.product_id
                where ml.account_no in ({','.join([str(x) for x in l['account_no'].tolist()])})
                ''', Mifosdb)
loclt['account_no'] = loclt['account_no'].astype('int64')
loclt

In [0]:
l = pd.merge(df, l, on = 'staff_code', how = 'inner')
loan_data = pd.merge(l, loclt, on = 'account_no', how = 'inner')
loan_data = loan_data[['staff_code', 'staff_name', 'client_id', 'client_name', 'account_no', 'product_name', 'closedon_date', 
                       'jan-23', 'feb-23', 'mar-23', 'apr-23', 'may-23', 'jun-23', 'average_monthly_balance']]

loan_data

In [0]:
declt = pd.read_sql_query(f'''
                select
                ml.client_id,
                mc.display_name as client_name,
                ml.account_no,
                mpl.name as product_name,
                ml.closedon_date
                from m_savings_account ml
                left join m_client mc on mc.id = ml.client_id
                left join m_savings_product mpl on mpl.id = ml.product_id
                where ml.account_no in ({','.join([str(x) for x in d['account_no'].tolist()])})
                ''', Mifosdb)
declt['account_no'] = declt['account_no'].astype('int64')
declt

In [0]:
d = pd.merge(df, d, on = 'staff_code', how = 'inner')
deposit_data = pd.merge(d, declt, on = 'account_no', how = 'inner')
deposit_data = deposit_data[['staff_code', 'staff_name', 'client_id', 'client_name', 'account_no', 'product_name', 'closedon_date', 
                       'jan-23', 'feb-23', 'mar-23', 'apr-23', 'may-23', 'jun-23', 'average_monthly_balance']]

deposit_data

In [0]:
loan = tst[0]

loan = duckdb.query(f'''
        select
        staff_code,
        sum(case when month = '2023_01' then total_value end) as jan_loan_value,
        sum(case when month = '2023_01' then total_count end) as jan_loan_count,
        
        sum(case when month = '2023_02' then total_value end) as feb_loan_value,
        sum(case when month = '2023_02' then total_count end) as feb_loan_count,
        
        sum(case when month = '2023_03' then total_value end) as mar_loan_value,
        sum(case when month = '2023_03' then total_count end) as mar_loan_count,
        
        sum(case when month = '2023_04' then total_value end) as apr_loan_value,
        sum(case when month = '2023_04' then total_count end) as apr_loan_count,
        
        sum(case when month = '2023_05' then total_value end) as may_loan_value,
        sum(case when month = '2023_05' then total_count end) as may_loan_count,
        
        sum(case when month = '2023_06' then total_value end) as jun_loan_value,
        sum(case when month = '2023_06' then total_count end) as jun_loan_count
        
        from loan
        group by 1
        ''').to_df()

loan

In [0]:
deposit = tst[1]

deposit = duckdb.query(f'''
        select
        staff_code,
        sum(case when month = '2023_01' then total_value end) as jan_loan_value,
        sum(case when month = '2023_01' then total_count end) as jan_loan_count,
        
        sum(case when month = '2023_02' then total_value end) as feb_loan_value,
        sum(case when month = '2023_02' then total_count end) as feb_loan_count,
        
        sum(case when month = '2023_03' then total_value end) as mar_loan_value,
        sum(case when month = '2023_03' then total_count end) as mar_loan_count,
        
        sum(case when month = '2023_04' then total_value end) as apr_loan_value,
        sum(case when month = '2023_04' then total_count end) as apr_loan_count,
        
        sum(case when month = '2023_05' then total_value end) as may_loan_value,
        sum(case when month = '2023_05' then total_count end) as may_loan_count,
        
        sum(case when month = '2023_06' then total_value end) as jun_loan_value,
        sum(case when month = '2023_06' then total_count end) as jun_loan_count
        
        from deposit
        group by 1
        ''').to_df()

deposit

In [0]:
final_deposit = pd.merge(df, deposit, on = 'staff_code', how = 'left')

final_deposit

In [0]:
final_loan = pd.merge(df, loan, on = 'staff_code', how = 'left')

final_loan

In [0]:
with pd.ExcelWriter("C:/Users/babatunde.omotayo/Downloads/V_bank_tasks/projects/staff performance/Staff financial performance Jan-Jun 2023.xlsx") as writer:
    deposit_data.to_excel(writer, sheet_name = 'deposit performance', index = False)
    loan_data.to_excel(writer, sheet_name = 'loan performance', index = False)

In [0]:
%%time

import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication

# Assuming you have multiple Pandas DataFrames named df1, df2, df3
dataframes = [loans, deposits]

# Create email content
msg = MIMEMultipart()
msg['From'] = 'babatunde.omotayo@vfdtech.ng'
msg['To'] = 'oluwatobi.odusanya@vfd-mfb.com'
msg['Subject'] = 'Loans & Deposit figures for 3PZKO'

# Attach each DataFrame as a separate CSV file to the email
for i, df in enumerate(dataframes, start=1):
    csv_data = df.to_csv(index=False)
    attachment = MIMEApplication(csv_data)
    attachment.add_header('Content-Disposition', 'attachment', filename=f'data{i}.csv')
    msg.attach(attachment)

# Send the email
smtp_server = 'smtp.office365.com'
smtp_port = 587
username = 'Babatunde.Omotayo@vfdtech.ng'
password = 'wmglfvqvnhdqgrrc'

with smtplib.SMTP(smtp_server, smtp_port) as server:
    server.starttls()
    server.login(username, password)
    server.sendmail(msg['From'], msg['To'], msg.as_string())